In [31]:
from typing import AnyStr
import os
os.chdir("..")
os.chdir("src")
from data_cleaning import DataCleaning
os.chdir("..")

FileNotFoundError: [Errno 2] No such file or directory: 'src'

In [ ]:
from bertopic import BERTopic
from bertopic.representation import KeyBERTInspired
from transformers import pipeline
import torch
from datasets import Dataset
from gensim.corpora import Dictionary
from gensim.models.coherencemodel import CoherenceModel

2024-04-11 19:29:31.030024: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-04-11 19:29:31.030301: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-11 19:29:31.032237: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-11 19:29:31.058006: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-11 19:29:31.758148: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warnin

## Sobre as métricas de avaliação
https://github.com/dice-group/Palmetto/issues/13

In [ ]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

# pipe = pipeline(
#     "zero-shot-classification",
#     model="MoritzLaurer/mDeBERTa-v3-base-xnli-multilingual-nli-2mil7",
#     framework="pt",
#     device=device,
#     batch_size=16,
# )


In [ ]:
data_cleaning_pipeline = DataCleaning()
df = data_cleaning_pipeline.run()

2024-04-11 19:29:34 - INFO - Data loaded!
2024-04-11 19:29:34 - INFO - Null values cleaned!
2024-04-11 19:29:34 - INFO - Data types asserted!
2024-04-11 19:29:34 - INFO - Full data cleaned!


## 

In [ ]:
data = Dataset.from_pandas(df)
docs = data["DS_OBJETO"]

In [ ]:
stop_words = []

with open("/home/brunodifranco/mestrado/mestrado-ufrgs-cmp617-tce/src/utils/stop_words.txt", "r") as file:
    for row in file:
        stop_words.append(row.strip())

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer_model = CountVectorizer(stop_words=stop_words)
# topic_model = BERTopic(vectorizer_model=vectorizer_model)

In [ ]:
topic_model = BERTopic(
    nr_topics="auto",
    embedding_model="all-MiniLM-L6-v2", 
    language="brazilian portuguese",
    min_topic_size=15,
    representation_model=KeyBERTInspired(),
    vectorizer_model=vectorizer_model

)

In [12]:
topics, probs = topic_model.fit_transform(docs)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

/home/brunodifranco/.local/lib/python3.10/site-packages/numba/np/ufunc/parallel.py:371: NumbaWarning: The TBB threading layer requires TBB version 2021 update 6 or later i.e., TBB_INTERFACE_VERSION >= 12060. Found TBB_INTERFACE_VERSION = 12050. The TBB threading layer is disabled.
  warnings.warn(problem)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process j

In [32]:
topics

{0: [('educação', 0.53414416),
  ('assistência', 0.44041756),
  ('sinalização', 0.42389134),
  ('agricultura', 0.42360923),
  ('construção', 0.41972032),
  ('escola', 0.419574),
  ('execução', 0.4119953),
  ('referência', 0.40376467),
  ('locação', 0.39269838),
  ('escolas', 0.39153555)],
 1: [('família', 0.5922185),
  ('medicamentos', 0.44498968),
  ('medicao', 0.4081061),
  ('projecao', 0.39517975),
  ('geriátricas', 0.36872268),
  ('geriatricas', 0.36872265),
  ('hospitalares', 0.3532731),
  ('panificacao', 0.33553988),
  ('pacientes', 0.31375098),
  ('medico', 0.3042565)],
 2: [('combustíveis', 0.5134498),
  ('combustiveis', 0.5134498),
  ('combustivel', 0.5078111),
  ('combustível', 0.50781107),
  ('diesel', 0.4961721),
  ('petróleo', 0.39581764),
  ('gasolina', 0.3122659),
  ('gás', 0.30902255),
  ('óleo', 0.294863),
  ('oléo', 0.29486293)],
 3: [('implantação', 0.6170585),
  ('implantes', 0.5991131),
  ('implante', 0.5695007),
  ('sinalização', 0.39835203),
  ('pavimentação', 0.

In [ ]:
import gensim.corpora as corpora
from gensim.models.coherencemodel import CoherenceModel

In [ ]:
cleaned_docs = topic_model._preprocess_text(docs)
vectorizer = topic_model.vectorizer_model
analyzer = vectorizer.build_analyzer()
tokens = [analyzer(doc) for doc in cleaned_docs]
dictionary = corpora.Dictionary(tokens)
corpus = [dictionary.doc2bow(token) for token in tokens]
topics = topic_model.get_topics()
topics.pop(-1, None)
topic_words = [
[word for word, _ in topic_model.get_topic(topic) if word != ""] for topic in topics
]
topic_words = [[words for words, _ in topic_model.get_topic(topic)] 
        for topic in range(len(set(topics))-1)][:10]

# Evaluate
coherence_model = CoherenceModel(topics=topic_words, 
                          texts=tokens, 
                          corpus=corpus,
                          dictionary=dictionary, 
                          coherence='c_v')
coherence = coherence_model.get_coherence()

In [35]:
topics_red = topic_model.reduce_topics(docs, nr_topics=5)

In [36]:
topics_red.get_topics()

{-1: [('administração', 0.5128263),
  ('construção', 0.48023286),
  ('elaboração', 0.47881675),
  ('sinalização', 0.47879952),
  ('educação', 0.47509193),
  ('pavimentação', 0.46780133),
  ('contratação', 0.4645041),
  ('realização', 0.46154442),
  ('assistência', 0.43208027),
  ('público', 0.42825383)],
 0: [('educação', 0.49767995),
  ('construção', 0.4613402),
  ('assistência', 0.45131466),
  ('referência', 0.42996985),
  ('alimentícios', 0.4158863),
  ('contratação', 0.4132297),
  ('pavimentação', 0.41275293),
  ('escola', 0.3991003),
  ('transporte', 0.39902392),
  ('administração', 0.38902014)],
 1: [('gráfico', 0.76230395),
  ('grafico', 0.76230395),
  ('rp', 0.23458505),
  ('', 0.2344533),
  ('', 0.2344533),
  ('', 0.2344533),
  ('', 0.2344533),
  ('', 0.2344533),
  ('', 0.2344533),
  ('', 0.2344533)],
 2: [('medicamentos', 0.99999994),
  ('medicamento', 0.9603776),
  ('médicos', 0.56999767),
  ('doces', 0.40482494),
  ('complexidade', 0.29081422),
  ('humanos', 0.19442846),
  

In [29]:
cleaned_docs = topics_red._preprocess_text(docs)
vectorizer = topics_red.vectorizer_model
analyzer = vectorizer.build_analyzer()
tokens = [analyzer(doc) for doc in cleaned_docs]
dictionary = corpora.Dictionary(tokens)
corpus = [dictionary.doc2bow(token) for token in tokens]
topics = topics_red.get_topics()
topics.pop(-1, None)
topic_words = [
[word for word, _ in topics_red.get_topic(topic) if word != ""] for topic in topics
]
topic_words = [[words for words, _ in topics_red.get_topic(topic)] 
        for topic in range(len(set(topics))-1)][:10]

# Evaluate
coherence_model = CoherenceModel(topics=topic_words, 
                          texts=tokens, 
                          corpus=corpus,
                          dictionary=dictionary, 
                          coherence='c_v')
coherence = coherence_model.get_coherence()

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

In [30]:
coherence

0.3720869663260227

In [22]:
cleaned_docs = topic_model._preprocess_text(docs)
vectorizer = topic_model.vectorizer_model
analyzer = vectorizer.build_analyzer()
tokens = [analyzer(doc) for doc in cleaned_docs]
dictionary = corpora.Dictionary(tokens)
corpus = [dictionary.doc2bow(token) for token in tokens]
topics = topic_model.get_topics()
topics.pop(-1, None)
topic_words = [
[word for word, _ in topic_model.get_topic(topic) if word != ""] for topic in topics
]
topic_words = [[words for words, _ in topic_model.get_topic(topic)] 
        for topic in range(len(set(topics))-1)][:10]

# Evaluate
coherence_model = CoherenceModel(topics=topic_words, 
                          texts=tokens, 
                          corpus=corpus,
                          dictionary=dictionary, 
                          coherence='c_v')
coherence = coherence_model.get_coherence()

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

In [23]:
coherence

0.4260404204330265

In [5]:
from utils.nlp import preprocess, remove_stop_words, stemmer_pt, lemma_pt
import nltk

In [6]:

# df = df.assign(
#     DS_OBJETO_NLP=df["DS_OBJETO"]
#     .apply(
#         lambda x: nltk.word_tokenize(x.lower(), language="portuguese")
#     )  # Tokenize
#     .apply(lambda x: [preprocess(word) for word in x])  # Other preprocessing
#     .apply(lambda x: list(filter(None, x)))  # Removes items with none
#     .apply(remove_stop_words)  # Removes stop words
#     .apply(
#         lambda x: [word for word in x if "rs" not in word]
#     )  # Remove tokens containing "rs" (which are cities)
# )


In [5]:
df_trial = df.sample(1000)

data = Dataset.from_pandas(df)
docs = data["DS_OBJETO"]

In [8]:
# from bertopic.vectorizers import ClassTfidfTransformer

# ctfidf_model = ClassTfidfTransformer(reduce_frequent_words=True)

In [7]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer_model = CountVectorizer(stop_words=stop_words)
# topic_model = BERTopic(vectorizer_model=vectorizer_model)

In [11]:
# zeroshot_topic_list = ["eletrica, hidraulica", 
#                        "escola, alimenticio, educacao", 
#                        "limpeza, obra, predio", 
#                        "rua, construcao, pavimentacao", 
#                        "saude, medico, familia",
#                        "servico, transporte, instalacao",
#                        "veiculos, maquinas, frota"]

# # We fit our model using the zero-shot topics
# # and we define a minimum similarity. For each document,
# # if the similarity does not exceed that value, it will be used
# # for clustering instead.
# topic_model = BERTopic(
#     embedding_model="all-MiniLM-L6-v2", 
#     language="brazilian portuguese",
#     min_topic_size=15,
#     zeroshot_topic_list=zeroshot_topic_list,
#     zeroshot_min_similarity=.4,
#     representation_model=KeyBERTInspired(),
#     vectorizer_model=vectorizer_model

# )
# topics, _ = topic_model.fit_transform(docs["DS_OBJETO"])

In [12]:
# topic_model.get_topic_info()

In [13]:
# topic_model.visualize_topics()

In [15]:
# cleaned_docs = topic_model._preprocess_text(docs["DS_OBJETO"])
# vectorizer = topic_model.vectorizer_model
# analyzer = vectorizer.build_analyzer()
# tokens = [analyzer(doc) for doc in cleaned_docs]
# dictionary = corpora.Dictionary(tokens)
# corpus = [dictionary.doc2bow(token) for token in tokens]
# topics = topic_model.get_topics()
# topics.pop(-1, None)
# topic_words = [
# [word for word, _ in topic_model.get_topic(topic) if word != ""] for topic in topics
# ]
# topic_words = [[words for words, _ in topic_model.get_topic(topic)] 
#         for topic in range(len(set(topics))-1)]

# # Evaluate
# coherence_model = CoherenceModel(topics=topic_words, 
#                             texts=tokens, 
#                             corpus=corpus,
#                             dictionary=dictionary, 
#                             coherence='c_v')
# coherence = coherence_model.get_coherence()

1839

In [ ]:
128753/

7

In [70]:
zeroshot_topic_list = ["eletrica, hidraulica", 
                       "Escola, Educação", 
                       "limpeza, obra, predio", 
                       "rua, construcao, pavimentacao", 
                       "saude, medico, familia",
                       "servico, transporte, instalacao",
                       "veiculos, maquinas, frota"]

min_topic_size = int(len(docs)/len(zeroshot_topic_list)*0.1) # Pelo menos 10% da divisao entre o total de observaçoes e o total de classes
# We fit our model using the zero-shot topics
# and we define a minimum similarity. For each document,
# if the similarity does not exceed that value, it will be used
# for clustering instead.
topic_model = BERTopic(
    # embedding_model="all-mpnet-base-v2", 
    # embedding_model="all-MiniLM-L6-v2", 
    embedding_model="thenlper/gte-small", 
    language="brazilian portuguese",
    min_topic_size=min_topic_size,
    zeroshot_topic_list=zeroshot_topic_list,
    zeroshot_min_similarity=.5,
    representation_model=KeyBERTInspired(),
    vectorizer_model=vectorizer_model
)
topics, _ = topic_model.fit_transform(docs)

In [ ]:
CoherenceModel()

In [71]:
topic_model.get_topic_info()

,Topic,Count,Name,Representation,Representative_Docs
0,0,50116,"rua, construcao, pavimentacao","[construção, pavimentação, administração, real...",[Contratação de empresa para a execução de obr...
1,1,32458,"servico, transporte, instalacao","[contratação, serviços, administração, serviço...",[CONTRATAÇÃO DE EMPRESA ESPECIALIZADA PARA PRE...
2,2,21647,"saude, medico, familia","[medicamentos, médicos, medico, médico, atendi...",[Registro de Preço para a futura e eventual aq...
3,3,12989,"Escola, Educação","[alimentação, alimentacao, alimentícios, alime...",[Eventual e futura aquisição de gêneros alimen...
4,4,8996,"veiculos, maquinas, frota","[pneus, veículos, veiculos, aquisição, proteto...",[AQUISIÇÃO DE PNEUS NOVOS PARA A FROTA DE VEÍC...
5,5,1308,"limpeza, obra, predio","[limpeza, limpa, limpador, aquisição, aquisiçõ...","[Aquisição de Material de Limpeza., Aquisição ..."
6,6,1239,"eletrica, hidraulica","[hidráulicas, hidraulica, hidráulica, hidrauli...","[AQUISIÇÃO DE UMA ESCAVADEIRA HIDRÁULICA, AQUI..."


In [9]:
zeroshot_topic_list = ["eletrica, hidraulica", 
                       "Escola, Educação", 
                       "limpeza, obra, predio", 
                       "rua, construcao, pavimentacao", 
                       "saude, medico, familia",
                       "servico, transporte, instalacao",
                       "veiculos, maquinas, frota"]

min_topic_size = int(len(docs)/len(zeroshot_topic_list)*0.1) # Pelo menos 10% da divisao entre o total de observaçoes e o total de classes
# We fit our model using the zero-shot topics
# and we define a minimum similarity. For each document,
# if the similarity does not exceed that value, it will be used
# for clustering instead.
topic_model = BERTopic(
    # embedding_model="all-mpnet-base-v2", 
    # embedding_model="all-MiniLM-L6-v2", 
    embedding_model="thenlper/gte-small", 
    language="brazilian portuguese",
    min_topic_size=min_topic_size,
    low_memory=True,
    # zeroshot_topic_list=zeroshot_topic_list,
    zeroshot_min_similarity=.7,
    representation_model=KeyBERTInspired(),
    vectorizer_model=vectorizer_model
)
topics, _ = topic_model.fit_transform(docs)

/home/bruno/mestrado-ufrgs/mestrado-ufrgs-cmp617-tce/.venv/lib/python3.10/site-packages/numba/np/ufunc/parallel.py:371: NumbaWarning: The TBB threading layer requires TBB version 2021 update 6 or later i.e., TBB_INTERFACE_VERSION >= 12060. Found TBB_INTERFACE_VERSION = 12050. The TBB threading layer is disabled.
  warnings.warn(problem)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/t

In [13]:
import pandas as pd
pd.set_option('display.max_colwidth', None)


In [11]:
cleaned_docs = topic_model._preprocess_text(docs)
vectorizer = topic_model.vectorizer_model
analyzer = vectorizer.build_analyzer()
tokens = [analyzer(doc) for doc in cleaned_docs]
dictionary = Dictionary(tokens)
corpus = [dictionary.doc2bow(token) for token in tokens]
topics = topic_model.get_topics()
topics.pop(-1, None)
topic_words = [
[word for word, _ in topic_model.get_topic(topic) if word != ""] for topic in topics
]
topic_words = [[words for words, _ in topic_model.get_topic(topic)] 
        for topic in range(len(set(topics))-1)]

# Evaluate
coherence_model = CoherenceModel(topics=topic_words, 
                            texts=tokens, 
                            corpus=corpus,
                            dictionary=dictionary, 
                            coherence='c_v')
coherence = coherence_model.get_coherence()
coherence

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

0.4153744865189709

,Topic,Count,Name,Representation,Representative_Docs
0,0,332,construction,"[empresas, empreitada, revitalização, paviment...","[Contratação de empresa, por regime de empreit..."
1,1,246,health/hospital,"[medicamentos, médico, referência, consultas, ...",[Constitui objeto da presente licitação o REGI...
2,2,195,education,"[educação, educacao, escolares, escolas, escol...",[Contratação de prestação de serviços de Trans...
3,3,120,food,"[alimentação, alimentacao, alimenticios, alime...",[AQUISIÇÃO DE GÊNEROS ALIMENTÍCIOS P/ALIMENTAÇ...
4,4,107,vehicles,"[referência, retroescavadeira, especificações,...",[Contratação de serviços de transporte com cam...


In [4]:
# https://stackoverflow.com/questions/75280182/topic-modelling-coherence-score

In [2]:
cleaning_pipeline = DataCleaning()
df = cleaning_pipeline.run()

2024-04-05 00:27:15 - INFO - Data loaded!
2024-04-05 00:27:15 - INFO - Null values cleaned!
2024-04-05 00:27:15 - INFO - Data types asserted!
2024-04-05 00:27:15 - INFO - Full data cleaned!


In [22]:
from bertopic import BERTopic


In [13]:
df_trial = df.sample(5000, random_state=42)

In [4]:
docs = df["DS_OBJETO"].values

In [5]:
topic_model = BERTopic(
    nr_topics=5,
    zeroshot_topic_list=[
        "vehicles",
        "construction",
        "health",
        "education",
        "food",
    ],
    min_topic_size=100,
    language="brazilian portuguese",
    low_memory=True,
)
topics, _ = topic_model.fit_transform(docs)

/home/bruno/mestrado-ufrgs/mestrado-ufrgs-cmp617-tce/.venv/lib/python3.10/site-packages/numba/np/ufunc/parallel.py:371: NumbaWarning: The TBB threading layer requires TBB version 2021 update 6 or later i.e., TBB_INTERFACE_VERSION >= 12060. Found TBB_INTERFACE_VERSION = 12050. The TBB threading layer is disabled.
  warnings.warn(problem)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/t

In [9]:
import pandas as pd

In [10]:
df["clusters"] = topics

In [11]:
df["clusters"].value_counts()

clusters
6    77159
5    41767
7     6000
8     2829
0      249
1      205
2      186
9      150
3      133
4       75
Name: count, dtype: int64

In [12]:
cleaned_docs = topic_model._preprocess_text(docs)
vectorizer = topic_model.vectorizer_model
analyzer = vectorizer.build_analyzer()
tokens = [analyzer(doc) for doc in cleaned_docs]
dictionary = corpora.Dictionary(tokens)
corpus = [dictionary.doc2bow(token) for token in tokens]
topics = topic_model.get_topics()
topics.pop(-1, None)
topic_words = [
[word for word, _ in topic_model.get_topic(topic) if word != ""] for topic in topics
]
topic_words = [[words for words, _ in topic_model.get_topic(topic)] 
        for topic in range(len(set(topics))-1)]

# Evaluate
coherence_model = CoherenceModel(topics=topic_words, 
                            texts=tokens, 
                            corpus=corpus,
                            dictionary=dictionary, 
                            coherence='c_v')
coherence = coherence_model.get_coherence()

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

In [13]:
coherence

0.532182199601841